In [ ]:
#EXAM 2025

# Question 1 – Segmentation des gumballs par Watershed
#1.a.i Prétraitement
#Chargement et affichage de l’image RGB
import cv2
import numpy as np
import matplotlib.pyplot as plt

gumballs = cv2.imread('gumballs.jpg')
gumballs_rgb = cv2.cvtColor(gumballs, cv2.COLOR_BGR2RGB)

plt.imshow(gumballs_rgb)
plt.title('Image RGB – Gumballs')
plt.axis('off')
plt.show()

#Choix du canal couleur
#Le canal rouge est le plus adapté car les gumballs rouges, oranges et roses y apparaissent avec une intensité élevée.
red_channel = gumballs_rgb[:,:,0]
plt.imshow(red_channel, cmap='gray')
plt.title('Canal rouge')
plt.axis('off')
plt.show()

#Lissage
#Un flou gaussien est utilisé pour réduire le bruit tout en conservant des contours lisses.
blur = cv2.GaussianBlur(red_channel, (5,5), 1)
plt.imshow(blur, cmap='gray')
plt.title('Image lissée (Gaussian Blur)')
plt.axis('off')
plt.show()

#Seuillage
_, binary = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
plt.imshow(binary, cmap='gray')
plt.title('Image binaire après seuillage')
plt.axis('off')
plt.show()

#Morphologie
#Ouverture pour supprimer le bruit + fermeture pour combler les trous.
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
clean = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
clean = cv2.morphologyEx(clean, cv2.MORPH_CLOSE, kernel)

plt.imshow(clean, cmap='gray')
plt.title('Image binaire nettoyée')
plt.axis('off')
plt.show()


# 1.a.ii Segmentation par Watershed
#Détection des graines
dist = cv2.distanceTransform(clean, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist, 0.5*dist.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

plt.imshow(sure_fg, cmap='gray')
plt.title('Graines pour Watershed')
plt.axis('off')
plt.show()

#Application du Watershed
unknown = cv2.subtract(clean, sure_fg)
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown==255] = 0

markers = cv2.watershed(gumballs, markers)

#Résultat final
result = gumballs_rgb.copy()
result[markers == -1] = [255,0,0]

plt.imshow(result)
plt.title('Résultat final Watershed')
plt.axis('off')
plt.show()

#1.b Discussion
#Le watershed est efficace pour séparer des objets en contact mais sensible au bruit et au mauvais choix de graines. Les variations de couleur et d’illumination compliquent la segmentation.

#Question 2 – Réalité augmentée : détection, suivi et remplacement
#2.a.i Détection par Template Matching
frame1 = cv2.imread('frame1.png')
frame1_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
template = cv2.imread('template.jpg',0)
frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

res = cv2.matchTemplate(frame1_gray, template, cv2.TM_SQDIFF)
min_val, _, min_loc, _ = cv2.minMaxLoc(res)
w, h = template.shape[::-1]

bbox = frame1_rgb.copy()
cv2.rectangle(bbox, min_loc, (min_loc[0]+w, min_loc[1]+h), (255,0,0), 2)
plt.imshow(bbox)
plt.title('Détection du damier')
plt.axis('off')
plt.show()

# 2.a.ii Suivi par Optical Flow
corners = cv2.goodFeaturesToTrack(frame1_gray, 36, 0.01, 5)

frame2 = cv2.imread('frame2.png')
frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

lk_params = dict(winSize=(15,15), maxLevel=2,
criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

p2, st, err = cv2.calcOpticalFlowPyrLK(frame1_gray, frame2_gray, corners, None, **lk_params)
# Choix des paramètres : fenêtre modérée pour déplacements lents, pyramide pour gérer les variations d’échelle

#2.a.iii Remplacement du damier
graphic = cv2.imread('graphic.jpg')
graphic_rgb = cv2.cvtColor(graphic, cv2.COLOR_BGR2RGB)
graphic_resized = cv2.resize(graphic_rgb, (w,h))


frame2_rgb = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
frame2_rgb[min_loc[1]:min_loc[1]+h, min_loc[0]:min_loc[0]+w] = graphic_resized


plt.imshow(frame2_rgb)
plt.title('Réalité augmentée – Damier remplacé')
plt.axis('off')
plt.show()

#Question 3 – CNN et ResNet50
#3.a CNN CIFAR-10 avec Early Stopping
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train,x_test = x_train/255.0, x_test/255.0
y_train,y_test = to_categorical(y_train,10), to_categorical(y_test,10)

model = Sequential([
Conv2D(8,(5,5),activation='relu',input_shape=(32,32,3)),
MaxPooling2D((2,2)),
Conv2D(16,(5,5),activation='relu'),
MaxPooling2D((2,2)),
Flatten(),
Dense(512,activation='relu'),
Dense(10,activation='softmax')
])

model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

early = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=2, restore_best_weights=True)

model.fit(x_train,y_train,epochs=3,batch_size=64,
validation_data=(x_test,y_test),callbacks=[early])

#3.b ResNet50 – Images de mains
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

model = ResNet50(weights='imagenet')
model.summary()

def preprocess_img(path):
img = image.load_img(path, target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
return preprocess_input(img)

img1 = preprocess_img('hand1.jpg')
img2 = preprocess_img('hand2.jpg')

pred1 = decode_predictions(model.predict(img1), top=3)[0]
pred2 = decode_predictions(model.predict(img2), top=3)[0]
#Discussion : Les prédictions peuvent être peu pertinentes car les mains ne font pas partie des classes ImageNet.

#3.c Fine-tuning (théorie)
#   Geler les premières couches
#   Ré-entraîner les couches finales
#   Ajuster le learning rate
#   Plus le dataset cible est différent, plus il faut dégeler de couches